# Issue Hospital Credentials
## Role: NHS Trust

You should have initialised your NHS Trust agent so that it has the public keys necessary to issue Hospital credentials.

In this notebook you will connect with each of the hospital agents and issue them a Verifiable Credential attesting that they are a hospital within your Trust.

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://nhstrust-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`
